<a href="https://colab.research.google.com/github/Nejel/neuralburger/blob/master/NeuralBurger_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Генерируем нейробугурты на рекуррентных нейронных сетях

Этот туториал демонстрирует генерацию текста на примере постов из паблика "Бугурт-Тред" во Вконтакте. История постов паблика используется как исходный корпус текстов для тренировки.

Тексты имеют следующий вид: 

<pre>
СЕТАП
@ 
РАЗВИТИЕ 
@
ПАНЧЛАЙН
</pre>


Модель тренируется на предсказание следующего символа (буквы или пробела) в последовательностях из 200 символов. Более длинные последовательности символов генерируются повторным вызовом модели. 

Этот туториал основан на туториале RNN text generation от Google. Ссылки на оригинальный тоториал:

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Оригинальный туториал от Google на сайте TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Он же в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Он же на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачать ipynb</a>
  </td>
</table>



While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.


## Setup

### Import TensorFlow and other libraries

In [0]:
import tensorflow as tf
import numpy as np
import os
import time

### Загрузим исходный файл с данными для обучения


In [0]:
# Для удобства я загрузил датасет с бугуртами на Гитхаб и обращаюсь к нему через raw.githubusercontent
path_to_file = tf.keras.utils.get_file('BT.txt', 'https://raw.githubusercontent.com/Nejel/datasets/master/BT.txt') 

### Чтение данных

In [7]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 3160845 characters


In [8]:
#Make burgers at their normal form
import re
text = re.sub(r"\n@  |\n@ ", r"@\n\r", text)

# Take a look at the first 250 characters in text
print(text[:250])

ПЛАНИРУЕТЕ СТОЛ НА НГ 
@
НЕТ ТЫ ЧТО, ЭТОГО МАЛО БУДЕТ, ДАВАЙ ЕЩЕ САЛАТОВ И ГОРЯЧЕГО 
@
2 ЯНВАРЯ 
@
ВЕСЬ СТОЛ СТОИТ ПРАКТИЧЕСКИ ЦЕЛЫЙ, СЪЕДЕНО ОТ СИЛЫ ДВА САЛАТА И ТРЕТЬ ГОРЯЧЕГО 
@
3 ЯНВАРЯ 
@
КАЖЕТСЯ У ПОМОЙНЫХ КОТОВ СЕГОДНЯ БУДЕТ ПИР


In [9]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

391 unique characters


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [11]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '#' :   5,
  '$' :   6,
  '%' :   7,
  '&' :   8,
  "'" :   9,
  '(' :  10,
  ')' :  11,
  '*' :  12,
  '+' :  13,
  ',' :  14,
  '-' :  15,
  '.' :  16,
  '/' :  17,
  '0' :  18,
  '1' :  19,
  ...
}


In [12]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'ПЛАНИРУЕТЕ СТ' ---- characters mapped to int ---- > [161 157 146 159 154 162 165 151 164 151   2 163 164]


### Задача предсказания следующего символа

Имея последовательность символов, какой наиболее вероятный следующий символ? Это задача, для которой мы будем оптимизировать наш алгоритм. На ввод в модель будет подаваться последовательность символов и модель будет на каждом шаге возвращать следующий за этой последовательностью символ.

### Создание тренировочных примеров и целей

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [13]:
# The maximum length sentence we want for a single input in characters
seq_length = 200
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

П
Л
А
Н
И


The `batch` method lets us easily convert these individual characters to sequences of the desired size.

In [14]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'ПЛАНИРУЕТЕ СТОЛ НА НГ \r\n\r@\n\rНЕТ ТЫ ЧТО, ЭТОГО МАЛО БУДЕТ, ДАВАЙ ЕЩЕ САЛАТОВ И ГОРЯЧЕГО \r\n\r@\n\r2 ЯНВАРЯ \r\n\r@\n\rВЕСЬ СТОЛ СТОИТ ПРАКТИЧЕСКИ ЦЕЛЫЙ, СЪЕДЕНО ОТ СИЛЫ ДВА САЛАТА И ТРЕТЬ ГОРЯЧЕГО \r\n\r@\n\r3 ЯНВАРЯ'
' \r\n\r@\n\rКАЖЕТСЯ У ПОМОЙНЫХ КОТОВ СЕГОДНЯ БУДЕТ ПИР \r\n\r@\n\rЭТО ТОЛЬКО КАЖЕТСЯ \r\n\r@\n\r4 ЯНВАРЯ \r\n\r@\n\rСАЛАТНОЕ МЕНЮ НА ЗАВТРАК, ОБЕД И УЖИН \r\n\r@\n\r5 ЯНВАРЯ \r\n\r@\n\r6 ЯНВАРЯ \r\n\r@\n\rВЕЧЕРОМ ТОГО ЖЕ ДНЯ \r\n\r@\n\rВТОРО'
'Й ДЕНЬ ПОДРЯД В АВТОБУСЕ ПОПАДАЕТСЯ СЧАСТЛИВЫЙ БИЛЕТИК \r\n\r@\n\rНО НИЧЕГО НЕ ПРОИСХОДИТ \r\n\r@\n\rТРЕТИЙ ДЕНЬ ПОДРЯД ПОПАДАЕТСЯ СЧАСТЛИВЫЙ БИЛЕТИК \r\n\r@\n\rПОТЕРЯЛ НАУШНИКИ\r\n\r@\n\rНАЗЫВАЕТ СЕБЯ ЛИБЕРАЛОМ \r\n\r@\n\rЗНА'
'ЧИТЕЛЬНАЯ ЧАСТЬ ПРОГРАММЫ - ВСЯЧЕСКИЕ ОГРАНИЧЕНИЯ ДЛЯ МИГРАНТОВ\r\n\r@\n\rПОМОГ ЕОТОВОЙ ПОДНЯТЬ УПАВШУЮ РУЧКУ \r\n\r@\n\r"ПИЗДОЛИЗ, ПОДКАБЛУЧНИК" \r\n\r@\n\rЕРОХИН ОТКРОВЕННО ЛЬСТИТ ЕЙ, ИСПОЛНЯЕТ КАКИЕ-ТО ТУПЫЕ ПОРУЧ'
'ЕНИЯ \r\n\r@\n\r"МОЛОДЕЦ, ВАНЬКА, НАСТОЯЩИЙ МУЖИК"\r\n\r@\n\rПОВЗРОСЛЕЛ \r\n\r@\n\r

For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [16]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'ПЛАНИРУЕТЕ СТОЛ НА НГ \r\n\r@\n\rНЕТ ТЫ ЧТО, ЭТОГО МАЛО БУДЕТ, ДАВАЙ ЕЩЕ САЛАТОВ И ГОРЯЧЕГО \r\n\r@\n\r2 ЯНВАРЯ \r\n\r@\n\rВЕСЬ СТОЛ СТОИТ ПРАКТИЧЕСКИ ЦЕЛЫЙ, СЪЕДЕНО ОТ СИЛЫ ДВА САЛАТА И ТРЕТЬ ГОРЯЧЕГО \r\n\r@\n\r3 ЯНВАР'
Target data: 'ЛАНИРУЕТЕ СТОЛ НА НГ \r\n\r@\n\rНЕТ ТЫ ЧТО, ЭТОГО МАЛО БУДЕТ, ДАВАЙ ЕЩЕ САЛАТОВ И ГОРЯЧЕГО \r\n\r@\n\r2 ЯНВАРЯ \r\n\r@\n\rВЕСЬ СТОЛ СТОИТ ПРАКТИЧЕСКИ ЦЕЛЫЙ, СЪЕДЕНО ОТ СИЛЫ ДВА САЛАТА И ТРЕТЬ ГОРЯЧЕГО \r\n\r@\n\r3 ЯНВАРЯ'


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [17]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 161 ('П')
  expected output: 157 ('Л')
Step    1
  input: 157 ('Л')
  expected output: 146 ('А')
Step    2
  input: 146 ('А')
  expected output: 159 ('Н')
Step    3
  input: 159 ('Н')
  expected output: 154 ('И')
Step    4
  input: 154 ('И')
  expected output: 162 ('Р')


### Создание тренировочных пакетов (batch)

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [18]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 200), (64, 200)), types: (tf.int64, tf.int64)>

## Построение модели

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
# #Базовая и самая простая модель для генерации текстов. 
# #Сейчас закомментирована в пользу более эффективных моделей (см. ниже)

# def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
#   model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(vocab_size, embedding_dim,
#                               batch_input_shape=[batch_size, None]),
#     tf.keras.layers.GRU(rnn_units,
#                         return_sequences=True,
#                         stateful=True,
#                         recurrent_initializer='glorot_uniform'),
#     tf.keras.layers.Dense(vocab_size)
#   ])
#   return model

In [0]:
# Возможно лучшая модель для генерации. 
# Достаточно простая чтобы быстро обучаться, но достаточно эффективная для генерации связанного текста

MODEL_VERSION = 0.03

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    # embedding layer
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    # GRU layer
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    
    # dropout layer
    tf.keras.layers.Dropout(0.4),
    
    # GRU layer
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    # dense layer
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

# Билд модели

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU (or LSTM) one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_training.png?raw=1)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [23]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 200, 391) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           100096    
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dropout (Dropout)            (64, None, 1024)          0         
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense (Dense)                (64, None, 391)           400775    
Total params: 14,140,551
Trainable params: 14,140,551
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [26]:
sampled_indices

array([330, 144,  45, 283, 100,  62,  24, 251,  88, 102,  82, 232, 347,
       382, 380,  29, 181,   4,  73, 307, 216, 253,  27, 172, 125, 303,
       355,  39, 339,  54, 139, 138, 383, 163, 247,  58, 205,  93, 214,
       296, 276,  16, 107, 116,  20, 357,  87, 327,  56, 134, 154, 227,
       252, 358, 181, 218,  83, 214, 274,  48, 294, 383, 155, 354, 344,
       152, 345, 197, 101, 105,  23, 112, 340, 224, 194, 313,  92, 133,
       345, 152,  75, 156, 267, 385,  50,  20, 366,  61,  30, 346, 140,
       355,  33,  78, 148, 186, 384, 362,  87, 195, 178, 240,  81, 137,
       320, 168, 364,  22, 172, 339, 325, 266,   1, 336,  90, 328,  65,
       122, 251, 179, 313,  92, 220, 131, 162, 247, 175, 183, 345,  62,
        37, 275,  29, 229,  43, 339,  25, 295,  34,  68, 223, 115, 248,
        50, 250, 381, 122,  51,  25, 250, 341, 142, 123, 241, 259, 385,
       107, 147, 264,  85,  76, 166, 244, 171, 282, 316,  88, 277, 385,
       287, 389, 336, 113, 333, 118, 365, 267,  57, 331, 158, 19

Decode these to see the text predicted by this untrained model:

## Посмотрим, что возвращает модель ещё не будучи обученной

In [27]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'Л САБАТОН \r\n\r@\n\rТЕПЕРЬ ТЫ И КАССИРША - БЕЙСБОЛЬНАЯ КОМАНДА \r\n\r@\n\rТЫ СТОИШЬ НА ПЕРВОЙ БАЗЕ, МАСТЕРСКИ ПРИНИМАЯ ВСЕ ПОДАЧИ ОТ ПИТЧЕРА \r\n\r@\n\rДАЖЕ НЕБО, ДАЖЕ УПРАВЛЯЮЩИЙ ПОРАЖАЮТСЯ ВАШЕЙ СЛАЖЕННОЙ РАБОТЕ '

Next Char Predictions: 
 'をІM„°^6حx¿rכヽ💬💥;г"i、Ѥد9Ъ̫✓気GヒVλδ🔥СاZы}Ѣ▆ყ.ÓĐ2画wまX͡Иזخ腹гѪsѢ༼P▃🔥Й対ロЖ・у»É5õペבрき|͜・ЖkКن😆R2！]=ーξ気AnВи😃開wсаפqβそЦ鹿4Ъヒのم\rデzもa̠حбき|҉͗РاЭе・^E༽;טKヒ7▅BdאýبRت💩̠S7تポЁ̤קض😆ÓБكulФתЩ”ざx\u2002😆℅😡デöシ̆ﻉنYんМуЪכもп«В/国пъ͚М腹👈ポСyеЛ'


## Обучим получившуюся модель

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Добавим функцию оптимизации (optimizer) и функцию потерь (loss function)

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [28]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 200, 391)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.9686337


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [0]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Выполнение обучения

To keep training time reasonable, use 10 to 30  epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [0]:
EPOCHS = 25

In [32]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/25
241/241 [==============================] - 57s 237ms/step - loss: 2.8706
Epoch 2/25
241/241 [==============================] - 57s 236ms/step - loss: 2.0900
Epoch 3/25
241/241 [==============================] - 57s 238ms/step - loss: 1.8046
Epoch 4/25
241/241 [==============================] - 58s 239ms/step - loss: 1.6526
Epoch 5/25
241/241 [==============================] - 57s 237ms/step - loss: 1.5661
Epoch 6/25
241/241 [==============================] - 57s 237ms/step - loss: 1.5088
Epoch 7/25
241/241 [==============================] - 57s 237ms/step - loss: 1.4662
Epoch 8/25
241/241 [==============================] - 57s 237ms/step - loss: 1.4322
Epoch 9/25
241/241 [==============================] - 57s 237ms/step - loss: 1.4033
Epoch 10/25
241/241 [==============================] - 57s 238ms/step - loss: 1.3793
Epoch 11/25
241/241 [==============================] - 58s 239ms/step - loss: 1.3568
Epoch 12/25
241/241 [==============================] - 57s 237ms/step - lo

## Теперь, когда модель обучена, можно попробовать сгенерировать новый текст

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [0]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.


![To generate text the model's output is fed back to the input](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_sampling.png?raw=1)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [0]:
temperature = 0.47

def generate_text(model, temperature, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 250

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = temperature

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))



### Вызовем функцию генерации текста


In [40]:
print(generate_text(model, temperature, start_string=u"И ТУТ ТЁТЯ ЙОБА ГОВОРИТ: "))

И ТУТ ТЁТЯ ЙОБА ГОВОРИТ: 
@
А ТЫ И НЕ ПРОТИВ 
@
ЗАХОДИШЬ В МАГАЗИН, СЛЫШИШЬ ПАРУ СТРАНЫ 
@
ДАЖЕ НЕБО, ДАЖЕ АЛЛАХ ВОСХИЩАЮТСЯ ТВОЕЙ ЕОТ 
@
ПРИХОДИТ ДЕД ЗА ВСЕМ ЛЕТОМ 
@
И ВОТ НАСТАЛ ДИАЛОГ 
@
НА СЛЕДУЮЩИЙ ДЕНЬ ПОД НИМ ПОЛУЧИЛОСЬ 
@
ЗАХОДИТ МАМКА 
@
ТЕБ


## Теперь попробуем генерировать нейробургеры массово

#### Подключим наш гугл-диск (работает только в google colab): 

https://colab.research.google.com/drive/1Br_C4odVdH3AjBohKXSvfQ1kif79aHSd?usp=sharing

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
count_of_burgers = 5 #count will be miltiplied by count of lines in bugstarts

bugstarts = ('И ТУТ ТЁТЯ ЙОБА ГОВОРИТ: ', 
             'ВСТРЕЧАЕШЬСЯ С ТЯН ', 
             'АНИМЕ НА АВЕ ', 
             'СМОТРИШЬ НОВОСТИ О КОРОНАВИРУСЕ ', 
             'ЕОТОВА ЗНАКОМИТСЯ С ТВОИМ БАТЕЙ ')

for i in range (count_of_burgers):
  print(i+1, " out of ", count_of_burgers)
  for i in range(len(bugstarts)):
    results = open('/content/drive/My Drive/Colab Notebooks/Results-26-04-2020-part2-bt.txt', 'a')
    lines = (generate_text(model, temperature, start_string=bugstarts[i]), "\n\n", "temperature: ", str(temperature), "  EPOCHS: ", str(EPOCHS), 
             " MODEL_VERSION: ", str(MODEL_VERSION), " BATCH_SIZE: ", str(BATCH_SIZE), 
             " BUFFER_SIZE: ", str(BUFFER_SIZE), " embedding_dim: ", str(embedding_dim), 
             " rnn_units: ", str(rnn_units), " seq_length: ", str(seq_length), "\n\n\n\n\n")
    results.writelines(lines)
    results.close()

1  out of  5
2  out of  5
3  out of  5
4  out of  5
5  out of  5


The easiest thing you can do to improve the results it to train it for longer (try `EPOCHS=30`).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.